In [105]:
# Importing libraries
import pandas as pd
import numpy as np
import openpyxl
from openpyxl import load_workbook
from datetime import datetime
import os
import re

## Step 1: Data Loading

In [106]:
os.listdir()
lista=[]
for i in os.listdir():
    if i.endswith('.xlsx'):
        lista.append(i)

print(lista)

['06-2024-Relatorio-de-comissao_Valeria Mendes_completo.xlsx', '06-2024-Relatorio-de-comissao_Luciano Costa_completo.xlsx', '06-2024-Relatorio-de-comissao_Guilherme Soares_completo.xlsx']


In [107]:
# This code will be used in order to hide sensitive information according to the GDPR
file = lista[2]
df1 = pd.read_excel(file)
# Accessing the wanted information
seller = df1.at[0, 'Unnamed: 1']

# Substitute the middle part of the CPF with asterisks
seller = re.sub(r'(\d{3})\.\d{3}\.\d{3}-(\d{2})', r'\1.***.***-\2', seller)
df1.at[0, 'Unnamed: 1'] = seller
# Assign the modified seller back to the dataframe
df1['SELLER'] = seller
df1.head(15)

,RELATÓRIO DE COMISSÕES,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,SELLER
0,Parceiro:,GUILHERME SOARES (148.***.***-64),NaN,NaN,NaN,NaN,GUILHERME SOARES (148.***.***-64)
1,Comissão:,15% (De 01/01/2020 até 31/12/2021),NaN,NaN,NaN,NaN,GUILHERME SOARES (148.***.***-64)
2,Contato:,guilherme@maxbot.com.br,NaN,NaN,NaN,NaN,GUILHERME SOARES (148.***.***-64)
3,Período:,De 01/01/2018 até 30/06/2024,NaN,NaN,NaN,NaN,GUILHERME SOARES (148.***.***-64)
4,Total:,R$ 32706.31,NaN,NaN,NaN,NaN,GUILHERME SOARES (148.***.***-64)
5,DATA PAGTO,NaN,CLIENTE,PRODUTO,VALOR PAGO,VALOR COMISSÃO,GUILHERME SOARES (148.***.***-64)
6,NaN,NaN,NaN,NaN,NaN,NaN,GUILHERME SOARES (148.***.***-64)
7,22/01/2020,EDITORANAPOLEAO,CRISTIANE RIBEIRO,MKT,1450,217.5,GUILHERME SOARES (148.***.***-64)
8,24/03/2020,DSOFTSOLUTIONS,DARI JUNIOR,ATD,390,58.5,GUILHERME SOARES (148.***.***-64)
9,30/03/2020,FIANCA,GERALDO CARLOS GONCALVES SILVA,ATD,286,42.9,GUILHERME SOARES (148.***.***-64)


In [108]:
#df1.info()
#df1.summary()

## Step 2: Data Transformation

In [109]:
# Creating a new column with the seller's name information
seller = df1.at[0, 'Unnamed: 1'] #accessing the wanted information
df1['SELLER'] = seller

#Creating a new column with the evaluated period
period = df1.at[3, 'Unnamed: 1']
df1['PERIOD'] = period

#utilizing the correct line as column names
df1.columns = df1.iloc[5]
df1.drop(df1.index[:7], inplace = True)

#Deleting a useless column 'VALOR COMISSÃO'
df1.drop('VALOR COMISSÃO', axis=1, inplace = True)

#renaming the second column as USERNAME instead of NaN
df1 = df1.rename(columns={np.nan: 'USERNAME'})
#renaming the index column


#reseting the index column
df1.reset_index(drop = True, inplace = True)
#Removing index name
df1 = df1.rename_axis(None, axis=1)

In [110]:
#Converting the data type of the 'DATA PAGTO' column to a date.

#Joining the two following steps into one.
#df['DATA PAGTO'] = pd.to_datetime(df['DATA PAGTO'], format='%d/%m/%Y')
#df['DATA PAGTO'] = df['DATA PAGTO'].dt.strftime('%Y/%m')

df1['DATA PAGTO'] = pd.to_datetime(df1['DATA PAGTO'], format='%d/%m/%Y').dt.strftime('%Y/%m')
df1.head()

,DATA PAGTO,USERNAME,CLIENTE,PRODUTO,VALOR PAGO,GUILHERME SOARES (148.***.***-64),De 01/01/2018 até 30/06/2024
0,2020/01,EDITORANAPOLEAO,CRISTIANE RIBEIRO,MKT,1450,GUILHERME SOARES (148.***.***-64),De 01/01/2018 até 30/06/2024
1,2020/03,DSOFTSOLUTIONS,DARI JUNIOR,ATD,390,GUILHERME SOARES (148.***.***-64),De 01/01/2018 até 30/06/2024
2,2020/03,FIANCA,GERALDO CARLOS GONCALVES SILVA,ATD,286,GUILHERME SOARES (148.***.***-64),De 01/01/2018 até 30/06/2024
3,2020/04,CENTROAVANCADO,CENTRO AVANCADO,MKT,920,GUILHERME SOARES (148.***.***-64),De 01/01/2018 até 30/06/2024
4,2020/04,DIEGODMCREDI,DIEGO SILVA,ATD,337,GUILHERME SOARES (148.***.***-64),De 01/01/2018 até 30/06/2024


In [111]:
#Obtaining the "MRR Agenda" by client
table1 = pd.pivot_table(df1, values = 'VALOR PAGO', index = 'USERNAME', columns = 'DATA PAGTO', aggfunc= 'sum', fill_value = 0)
table1

DATA PAGTO,2020/01,2020/03,2020/04,2020/05,2020/06,2020/07,2020/08,2020/09,2020/10,2020/11,...,2023/09,2023/10,2023/11,2023/12,2024/01,2024/02,2024/03,2024/04,2024/05,2024/06
USERNAME,,,,,,,,,,,,,,,,,,,,,
334701,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,350.0,1933.2,0.0,0.0,0.0,0.00,0.0
01,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
3dcure,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,547.0,547.0,630.16,617.0
ADRISI,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
AVUNHAS,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
vtrans,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,376.9,500.00,500.0
walmir,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
worldrio,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,4800.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0


In [112]:
#Adding a column to obtain the number of months each client has made a payment
table1['NUM_PAYMENTS'] = (table1>0).sum(axis=1)
table1

DATA PAGTO,2020/01,2020/03,2020/04,2020/05,2020/06,2020/07,2020/08,2020/09,2020/10,2020/11,...,2023/10,2023/11,2023/12,2024/01,2024/02,2024/03,2024/04,2024/05,2024/06,NUM_PAYMENTS
USERNAME,,,,,,,,,,,,,,,,,,,,,
334701,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,350.0,1933.2,0.0,0.0,0.0,0.00,0.0,5
01,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,1
3dcure,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,547.0,547.0,630.16,617.0,4
ADRISI,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,7
AVUNHAS,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
vtrans,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,376.9,500.00,500.0,3
walmir,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,3
worldrio,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4800.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,1


In [113]:
#A function to get a list with X months prior to current month.
def last_months(x):
    now = pd.Timestamp(datetime.now())
    dates = []
    for i in range (x):
        date = now - pd.DateOffset(months=i+1)
        year_month = date.strftime('%Y/%m')
        dates.append(year_month)
    return dates

In [114]:
#Just checking if our function gets the results we need
last_month = (last_months(1))
last_3_months = last_months(3)
type((last_month))
print(last_3_months)

['2024/06', '2024/05', '2024/04']


In [115]:
#Creating a column that gives us the number of payments in the last three months.
#This is important so we do not get fooled by clients that buys 6 or 12 months for example.
table1['NUM_PAYMENTS_LAST3_MON'] = table1.loc[:,last_3_months].gt(0).sum(axis=1)

In [116]:
table1.sort_values(by='NUM_PAYMENTS_LAST3_MON', ascending=False).head(300)

DATA PAGTO,2020/01,2020/03,2020/04,2020/05,2020/06,2020/07,2020/08,2020/09,2020/10,2020/11,...,2023/11,2023/12,2024/01,2024/02,2024/03,2024/04,2024/05,2024/06,NUM_PAYMENTS,NUM_PAYMENTS_LAST3_MON
USERNAME,,,,,,,,,,,,,,,,,,,,,
eumarfabiano,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,767.0,807.65,849.04,850.14,827.0,849.04,893.06,888.46,9,3
namare,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,203.50,325.60,325.60,325.6,325.60,342.50,342.50,7,3
rodolfofelipefm,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,380.0,380.00,380.00,380.00,380.0,380.00,399.72,399.72,15,3
gruporevest,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,677.0,677.00,677.00,677.00,677.0,752.21,780.51,780.51,12,3
renatawmacademy,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,877.0,877.00,877.00,877.00,877.0,877.00,1066.02,1057.52,30,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
atendimentosefin,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,15,0
baumann01,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,3,0
beegrafica,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,437.00,451.44,0.00,0.0,0.00,0.00,0.00,2,0


In [117]:
table1

DATA PAGTO,2020/01,2020/03,2020/04,2020/05,2020/06,2020/07,2020/08,2020/09,2020/10,2020/11,...,2023/11,2023/12,2024/01,2024/02,2024/03,2024/04,2024/05,2024/06,NUM_PAYMENTS,NUM_PAYMENTS_LAST3_MON
USERNAME,,,,,,,,,,,,,,,,,,,,,
334701,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,350.0,1933.2,0.0,0.0,0.0,0.00,0.0,5,0
01,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,1,0
3dcure,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,547.0,547.0,630.16,617.0,4,3
ADRISI,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,7,0
AVUNHAS,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
vtrans,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,376.9,500.00,500.0,3,3
walmir,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,3,0
worldrio,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,1,0


In [118]:
table1[(table1[last_month[0]]>0) & (table1['NUM_PAYMENTS'] <=3) & (table1['NUM_PAYMENTS_LAST3_MON'] <=3)]

DATA PAGTO,2020/01,2020/03,2020/04,2020/05,2020/06,2020/07,2020/08,2020/09,2020/10,2020/11,...,2023/11,2023/12,2024/01,2024/02,2024/03,2024/04,2024/05,2024/06,NUM_PAYMENTS,NUM_PAYMENTS_LAST3_MON
USERNAME,,,,,,,,,,,,,,,,,,,,,
centropsicanalise,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,557.0,568.88,557.00,3,3
chappa,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,557.0,557.00,557.00,3,3
ciaemporio,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,4678.80,1,1
cleversoncvs,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1900.0,0.0,0.0,0.0,0.0,0.00,1900.00,2,1
dba_grupodba,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,4560.00,2,1
funerariavip,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,557.00,557.00,2,2
kaz_formaturas,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,352.42,448.02,2,2
nuclearimobiliaria,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,557.29,617.00,2,2
ramsolution,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,4884.00,3,1


In [119]:
dict_map = {True: 'Elegible', False:'Ineligible'}
table1['COMISSION'] = ((table1['NUM_PAYMENTS'] <=3 ) & (table1['NUM_PAYMENTS'] == table1['NUM_PAYMENTS_LAST3_MON']) & (table1[last_month[0]] > 0)).map(dict_map)

In [120]:
table1 = table1.sort_values(by='COMISSION')
table1

DATA PAGTO,2020/01,2020/03,2020/04,2020/05,2020/06,2020/07,2020/08,2020/09,2020/10,2020/11,...,2023/12,2024/01,2024/02,2024/03,2024/04,2024/05,2024/06,NUM_PAYMENTS,NUM_PAYMENTS_LAST3_MON,COMISSION
USERNAME,,,,,,,,,,,,,,,,,,,,,
vtrans,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,376.9,500.00,500.00,3,3,Elegible
centropsicanalise,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,557.0,568.88,557.00,3,3,Elegible
chappa,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,557.0,557.00,557.00,3,3,Elegible
kaz_formaturas,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,352.42,448.02,2,2,Elegible
ciaemporio,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,4678.80,1,1,Elegible
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
asmengenharia,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,447.0,447.0,447.0,447.0,447.0,470.20,470.20,16,3,Ineligible
atendimentosefin,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,15,0,Ineligible
baumann01,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,3,0,Ineligible


# Comission Due

In [121]:
Comission_Value = (table1.loc[table1['COMISSION'] == 'Elegible', last_month[0]].sum())/10
Comission_Value

791.482

## Saving the table of interest in the excel file

In [122]:
with pd.ExcelWriter(file, engine = 'openpyxl', mode = 'a', if_sheet_exists = 'replace') as writer:
    table1.to_excel(writer, sheet_name='DadosComissoes')